In [2]:

import torch 
#dd


In [9]:
input = torch.tensor([0,1,2])
input,input.shape

(tensor([0, 1, 2]), torch.Size([3]))

In [ ]:
#lkkkk

In [4]:
from torch.nn import Embedding

embedding = Embedding(3,4)
embedded_sentence =embedding(input)
embedded_sentence,embedded_sentence.shape


(tensor([[-1.8719,  0.4048, -1.4430,  0.2267],
         [ 0.9684,  0.0665, -0.1522, -0.6881],
         [-0.1073, -0.5513, -1.7242,  0.9203]], grad_fn=<EmbeddingBackward0>),
 torch.Size([3, 4]))

In [5]:

q_vector=[]
k_vector=[]
v_vector=[]
q_vector.clear()
k_vector.clear()
v_vector.clear()

Wq = torch.rand(4,3)
Wk = torch.rand(4,3)
Wv = torch.rand(4,3)


for word in embedded_sentence:
    q_vector.append(torch.matmul(word.unsqueeze(0),Wq))
    k_vector.append(torch.matmul(word.unsqueeze(0),Wk))
    v_vector.append(torch.matmul(word.unsqueeze(0),Wv))


In [6]:
q_vector

[tensor([[-0.8496, -1.2055, -1.9319]], grad_fn=<MmBackward0>),
 tensor([[-0.1065,  0.2023,  0.1365]], grad_fn=<MmBackward0>),
 tensor([[-0.6184, -0.8655, -0.7872]], grad_fn=<MmBackward0>)]

In [7]:
k_vector , k_vector[0].shape

([tensor([[-1.5353, -1.0229, -2.5482]], grad_fn=<MmBackward0>),
  tensor([[-0.2971, -0.1036,  0.1708]], grad_fn=<MmBackward0>),
  tensor([[-1.1088, -1.5213, -1.3088]], grad_fn=<MmBackward0>)],
 torch.Size([1, 3]))

In [8]:
v_vector

[tensor([[-1.6836, -1.0530, -1.5812]], grad_fn=<MmBackward0>),
 tensor([[-0.2186, -0.4764, -0.0024]], grad_fn=<MmBackward0>),
 tensor([[-0.3542, -0.8676, -1.3166]], grad_fn=<MmBackward0>)]